## Activity 1: Convert nucleotide sequence to binary representation and output to file in little-endian ordering

#### Import 'bitarray' library to handle binary data

In [ ]:
from bitarray import bitarray

#### Define the sequence

In [ ]:
seq = "ACCGCATACTAG"

#### Create dictionary mapping a nucleotide letter to a bitarray

In [ ]:
# The bitarrays will later be inserted into a larger bitarray
bitD = {"A":bitarray('00'), 
        "C":bitarray('10'), 
        "G":bitarray('01'), 
        "T":bitarray('11')}

#### Create bitarray of of 0's of size: n nucleotides x 2

In [ ]:
# This will hold the entire bit representation of the sequence
seqbits1 = (len(seq)*2) * bitarray('0')
print(seqbits1)

#### Cycle through the sequence, setting bit-pairs from MSB to LSB

In [ ]:
# Set the offset of where we want to next insert a 2-bit array representation
offset = 0

# Cycle through nucleotide sequence
for x in seq:
    
    # lookup 2-bit array for nucleotide letter and insert into bits bitarray
    seqbits1[offset:offset+2] = bitD[x]
    
    # update the offset for next insertion
    offset+=2

# For sanity checking, print the final bit bitarray
print(seqbits1)

#### Create bitarray that is ordered as "little-endian". Note, while the declaration of the bitarray changes, the logic that proceeds remains the same:

In [ ]:
seqbits2 = (len(seq)*2) * bitarray('0', endian="little")

# Set the offset of where we want to next insert a 2-bit array representation
offset = 0

# Cycle through nucleotide sequence
for x in seq:
    
    # lookup 2-bit array for nucleotide letter and insert into bits bitarray
    seqbits2[offset:offset+2] = bitD[x]
    
    # update the offset for next insertion
    offset+=2

# The following statement will print bitarray as built up (but NOT in little-endian)
print(seqbits2)

#### To see the difference in the first 3 bytes (big-endian bit-ordering) and the last 3 bytes (little-endian ordering), we must write the bytes to file and view the file

In [ ]:
# Write the two bitarray to file. 
# This is where little-endian will be applied to seqbits2
with open("my_binary1.bin", "wb") as o:
    o.write(seqbits1.tobytes()) 
    o.write(seqbits2.tobytes()) 

#### You can view the file using xxd on the linux command line:
`xxd -b my_binary1.bin`

This returns

`00000000: 00101001 10001100 10110001 10010100 00110001 10001101  )...1.`

Where

`00000000:` refers to the address of the first byte that appear in the line in the file (in binary) (=0)

`00101001 10001100 10110001 10010100 00110001 10001101` are the first (and in our care the only) 6 bytes in the file. The first 3 bytes are the DNA sequence representation in "big-endian" bit ordering. The last bytes are the DNA sequence in "little-endian" bit-ordering.

`)...1.` is the ASCII translation of each byte (if can be translated to an ASCII character (otherwise `.`).



## Activity 2: read back the DNA sequence stored in the file in little-endian bit order

#### We will create a dictionary that maps decimal values of bit-pairs back to letters so we can recreate the original DNA sequence. 

In [ ]:
# Create an integer to nuclotide mapping using a dictionary
letterMap = {0:"A", 1:"C", 2:"G", 3:"T"}

#### Create variable to store a string of the original sequence:

In [ ]:
sequence = ""

#### Create a binary mask = 3 (binary representation `00000011`)

In [ ]:
# mask = 00000011
mask = 3  

#### Now we will open the file you created above and cycle through bytes 4 to 6. For each byte, we will apply the `&` binary operator with the mask to obtain the decimal representation for the next bit-pair in the byte that we can look up in the dictionary to determine the next letter in the DNA sequence.

In [ ]:
# Open binary file created above in binary mode: 
with open("my_binary1.bin", "rb") as f:
    
    # Go to the fourth byte in file (index = 3 as bytes addresses are zero-indexed): 
    f.seek(3)
    
    # Read the next 3 bytes that hold the bit-pairs in little-endian order that we will translate: 
    b = f.read(3)
    
    # Cycle through each byte returned from the read() function:
    for byte in b:
        
        # We know we will have to repeat the following 4 times as 4 letters per byte in our example
        for i in range(4):
            
            # Perform binary and (&) with the mask flagging the last two bits of the byte:
            genoBits = (byte & mask)
            
            # Convert the number using the dictionary above and add into numpy array. 
            sequence+= letterMap[genoBits]

            # perform two right shifts to move all bits two places to the right
            byte = byte >> 2

# Output the sequence.
print(sequence)


## Activity 3 - part 1:  handling incomplete bytes


In [ ]:
## Define the sequence
newSeq = "ACCGCATACTAGTA"

# Create dictionary mapping a nucleotide letter to a bitarray
# The bitarrays will later be inserted into a larger bitarray
bitD = {"A":bitarray('00'), 
        "C":bitarray('10'), 
        "G":bitarray('01'), 
        "T":bitarray('11')}

In [ ]:
# Create empty bitarray of size: n nucleotides x 2
# This will hold the entire bit representation of sequence
#seqbits3 = bitarray(len(newSeq)*2, endian='little')
seqbits3 = (len(newSeq)*2) * bitarray('0', endian="little")


# Set the offset of where we want to next insert a 2-bit array representation
offset = 0

# Cycle through nucleotide sequence
for x in newSeq:
    
    # lookup 2-bit array for nucleotide letter and insert into bits bitarray
    seqbits3[offset:offset+2] = bitD[x]
    
    # update the offset for next insertion
    offset+=2

In [ ]:
# Open file for writing in binary mode 
with open("my_binary2.bin", "wb") as o:
    
    # provide an integer (14) that represents the length of the string and store as first byte in file
    o.write(len(newSeq).to_bytes(1, byteorder="little", signed=False))
    
    # then write the bitarray
    o.write(seqbits3.tobytes()) 


### Activity 3 - part 2: read the file back to obtain sequence length to help decode the sequence from binary representation


In [ ]:
# Variable to store sequence
newSeqOut = ""
# Variable to keep track of number of nucleotides read.
pairsRead = 0

with open("my_binary2.bin", "rb") as f:
    
    # Get integer stored in first byte
    seqLength = int.from_bytes(f.read(1), byteorder="little")
    
    # Go to the second byte in file (index = 1): 
    f.seek(1)
    
    # Read the rest of file in: 
    b = f.read()
    
    # Cycle through each byte returned from the read() function:
    for byte in b:
        
        # We know we will have to repeat the following 4 times as 4 letters per byte in our example
        for i in range(4):
            
            # Perform binary and (&) with the mask flagging the last two bits of the byte:
            genoBits = (byte & mask)
            
            # Convert the number using the dictionary above and add into numpy array. 
            newSeqOut += letterMap[genoBits]

            # Update count on number of bit-pairs (nucleotides) read
            pairsRead += 1
            
            # check if we still need to continue - i.e. we have not obtained all nucleotides
            if pairsRead < seqLength:
                # perform two right shifts to move all bits two places to the right
                byte = byte >> 2
            else:
                # we're done!
                break 

print(newSeqOut)